# Instantiate API key and Distribution Center

For this public file I am required to leave out the distribution center for NDA purposes

In [1]:

API_key = ''
Dist_Center = '',




In [2]:
import googlemaps as gmap
import pandas as pd
import numpy as np

Creating a Master dataframe for total route miles

In [4]:

Master_Sheet = pd.DataFrame()

### Creating global variable for forecasted data delivery week 
For this problem we looked at 3 months of forecasted data that I created from historical demand

In [206]:
global week
week = 1

### Joining address data with demand data for specified week demand estimate

The address data is then correlated with its forecasted demand and used later in the VRP

Two main global variables created:
* a_dress - for addresses cleaned and prepped for google distance matrix api usage
* key - key for address node connection (used for master dataframe)

In [208]:
import pandas as pd
#
df2 = pd.read_csv('address_csv.csv', index_col = 'STORE_NUM')
Grouped_Demand_Total = pd.read_csv('', index_col = 'Store #')

#strip spaces out of lines
df2.ADDR_LINE1_TXT = df2.ADDR_LINE1_TXT.str.strip()
df2.CITY_NM = df2.CITY_NM.str.strip()
df2.ST_CD = df2.ST_CD.str.strip()

#Create new column with GOOGLEAPI add
df2['GoogleAPI_Address'] = df2.ADDR_LINE1_TXT+'+'+df2.CITY_NM+'+'+df2.ST_CD
df2['State'] = df2.ST_CD
df2.GoogleAPI_Address = df2.GoogleAPI_Address.str.lstrip()
df2.GoogleAPI_Address = df2.GoogleAPI_Address.replace(to_replace=' ', value='+', regex=True)

#join
Grouped_Demand_Total = Grouped_Demand_Total.join(df2.GoogleAPI_Address, how='left')
Grouped_Demand_Total = Grouped_Demand_Total.join(df2.State, how='left')
Grouped_Demand_Total = Grouped_Demand_Total.join(df2.LONG_ZIP_CD, how='left')
#------------------------------------------FOR TENNESSEE ONLY-------------------------------------------------------------###
Grouped_Demand_Total = Grouped_Demand_Total[Grouped_Demand_Total.State != 'TN']
Grouped_Demand_Total = Grouped_Demand_Total[~Grouped_Demand_Total.LONG_ZIP_CD.isin(Florida_Exclude)]
#------------------------------------------------------------------------------------------------------------------------###

#create Distance Matrix addresses
a_dress = list(Dist_Center,)
for i in Grouped_Demand_Total.GoogleAPI_Address[Grouped_Demand_Total.Week==week]:
    a_dress.append(i)
    
global a
a = a_dress
    
#Create Non Google Addresses
df2.LONG_ZIP_CD = df2.LONG_ZIP_CD.astype(str)
df2['Search'] = df2.ADDR_LINE1_TXT+','+' '+df2.CITY_NM+','+' '+df2.ST_CD+','+' '+df2.LONG_ZIP_CD
Grouped_Demand_Total = Grouped_Demand_Total.join(df2.Search, how='left')

b_dress = list(Dist_Center,)
for i in Grouped_Demand_Total.Search[Grouped_Demand_Total.Week == week]:
    b_dress.append(i)

    

global key
b = b_dress
key = pd.DataFrame(b)
key = key.rename(columns = {0:'Store_Address'})

# Distance Matrix Creation

* Google Distance Matrix API was used to pull distances to create matrix
* Matrix was then saved on harddrive so that I wouldn't go over the free number of requests


In [382]:
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import urllib.request
import pandas as pd
import numpy as np
import time

def create_data():
    """Creates the data."""
    data = {}
    #data['API_key'] = ''
    data['addresses'] = a
    return data

def create_distance_matrix(data):
    addresses = data["addresses"]
    API_key = data["API_key"]
    # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
    max_elements = 100
    num_addresses = len(addresses) # 16 in this example.
    # Maximum number of rows that can be computed per request (6 in this example).
    max_rows = max_elements // num_addresses
    # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
    q, r = divmod(num_addresses, max_rows)
    dest_addresses = addresses
    distance_matrix = []
    # Send q requests, returning max_rows rows per request.
    for i in range(q):
        origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)

    # Get the remaining remaining r rows, if necessary.
    if r > 0:
        origin_addresses = addresses[q * max_rows: q * max_rows + r]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)
    return distance_matrix

def send_request(origin_addresses, dest_addresses, API_key):
    def build_address_str(addresses):
        address_str = ''
        for i in range(len(addresses) - 1):
            address_str += addresses[i] + '|'
        address_str += addresses[-1]
        return address_str

    request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
    origin_address_str = build_address_str(origin_addresses)
    dest_address_str = build_address_str(dest_addresses)
    request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
    jsonResult = urllib.request.urlopen(request).read()
    global response
    response = json.loads(jsonResult)
    return response

def build_distance_matrix(response):
    distance_matrix = []
    for row in response['rows']:
        row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
        distance_matrix.append(row_list)
    return distance_matrix

########
# Main #
########
def main():
    """Entry point of the program"""
    # Create the data.
    data = create_data()
    addresses = data['addresses']
    API_key = data['API_key']
    distance_matrix = create_distance_matrix(data)
    global df
    df = distance_matrix
    
      

if __name__ == '__main__':
    main()


### Demand Input - distance matrix saved to harddrive

In [209]:

#Create Demand List for Weights
global actual_demand
actual_demand = list(Grouped_Demand_Total.Total_Weight[Grouped_Demand_Total.Week == week])
actual_demand.insert(0,0)
actual_demand = list(np.array(actual_demand)*-1) 

#TO CSV------------------------------------------------------------------------------------------SAVE MATRIX
csv = np.asarray(df)
df_csv = pd.DataFrame(csv)
df_csv.to_csv(''.format(str(week),Dist_Center), index=False)

### Distance Matrix pulled from harddrive - for iterations on number of trucks

In [210]:
global df1
df1 = pd.read_csv(''.format(str(week),Dist_Center))

df1 = np.asarray(df1)

# Main Program 

### **Modifications:**

1. First columns of distance matrix was transformed using numpy in order that trucks would not "return" to the depot. Because in wood products trucking, the company is only charged for miles *to* location.

2. Demand nodes are connected to forecasted demand automatically from the prior 'week' assignment (data['demands'] = acutal_demand)

3. Vehicle capacities are added as a constraint  - I estimated demand in pounds

4. Number of stops were iterated and modified to get a best and worst case total mileage

5. A Master_Info dictionary was created to later transform into a Pandas dataframe and then into a CSV file 

In [2]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    Me_to_Mi = 0.000621371
    df1[:,0:1] = 0 
    df2 = df1*Me_to_Mi
    data = {}
    data['distance_matrix'] = df2
    data['num_vehicles'] = 1
    data['depot'] = 0
    data['demands'] = actual_demand
    data['vehicle_capacities']= [44000]                                  
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    global total_load
    total_load = 0
    global total_distance
    total_distance = 0
    global route_distances
    route_distances =[]
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = data['vehicle_capacities'][vehicle_id]
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} miles\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        route_distances.append(route_distance)
        total_distance += route_distance
        total_load += data['vehicle_capacities'][vehicle_id] - route_load
        total_load = total_load
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {} miles'.format(max_route_distance))
    print('Total distance of all routes: {}miles'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
   
       




def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    

    def demand_callback(from_index):
    # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        False,  # start cumul to zero
        'Capacity')
    
    
    
    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        99999,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    count_dimension_name = 'count'
    # assume some variable num_nodes holds the total number of nodes
    routing.AddConstantDimension(
        1, # increment by one every time
        30,  # max value forces equivalent # of jobs
        True,  # set count to zero
        count_dimension_name)
    count_dimension = routing.GetDimensionOrDie(count_dimension_name)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 90

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    def get_routes(manager, routing, solution, num_routes):
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(num_routes):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes


    # Print solution on console.
    if solution:
        global master_info
        master_info = {}
        print_solution(data, manager, routing, solution)
        routes = get_routes(manager, routing, solution, data['num_vehicles'])
        # Display the routes.
        total_stops = 0
        for i, route in enumerate(routes):
            print('Route', i, (len(route)-2), route_distances[i])
            master_info['Route{}_Total_Miles'.format(i)] = route_distances[i]
            master_info['Route{}_Stops'.format(i)] = (len(route)-2)
            total_stops = (len(route)-2) + total_stops
        global r_sheet 
        r_sheet = {}
        count = 0
        for i in routes:
            r_sheet['route{}'.format(count)] = i
            count = count +1
        master_info['Total_Miles'] = total_distance
        master_info['Total_Load'] = total_load
        master_info['Total_Stops'] = total_stops
        master_info['Number_Vehicles'] = data['num_vehicles']
        master_info['Week'] = week
        
            
            
                
       
            
  
  


if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'ortools'

### Estimates 

Each iteration was added to the previously instantiated Master dataframe so that a CSV file of addresses and total mileage could be created

In [234]:
Master = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in master_info.items() ]))
Master_Sheet = Master_Sheet.append(Master, ignore_index = True, sort=True)
Master_Sheet.to_csv('', index=False)
Master_Sheet.tail(7)

,Number_Vehicles,Route0_Stops,Route0_Total_Miles,Route10_Stops,Route10_Total_Miles,Route1_Stops,Route1_Total_Miles,Route2_Stops,Route2_Total_Miles,Route3_Stops,...,Route7_Stops,Route7_Total_Miles,Route8_Stops,Route8_Total_Miles,Route9_Stops,Route9_Total_Miles,Total_Load,Total_Miles,Total_Stops,Week
125,8,17,342,NaN,NaN,10,295,5.0,349.0,0.0,...,12.0,348.0,NaN,NaN,NaN,NaN,37433.568453,1645,58,1
126,7,17,342,NaN,NaN,10,295,5.0,349.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1645,58,1
127,6,17,342,NaN,NaN,10,295,5.0,349.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1645,58,1
128,5,10,343,NaN,NaN,6,313,6.0,343.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1657,58,1
129,4,16,299,NaN,NaN,19,357,9.0,377.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1403,58,1
130,3,22,411,NaN,NaN,17,414,19.0,414.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1239,58,1
131,2,29,643,NaN,NaN,29,622,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37433.568453,1265,58,1




# Map CSV Creation

In [237]:
new_dict = {}
count = 0
old_list = list(key.Store_Address)
for i in r_sheet.values():
    new_list = list()
    for x in i:
        new_list.append(old_list[x])
    new_dict['route{}'.format(count)] = new_list
    count = count +1

new_routes = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in new_dict.items() ]))
new_routes.to_csv('C:\\Users\cuomo\OneDrive\GATech\CMPC  Internship\VRP\Distance_Matrix\Routes\Matrix_Week{}_Trucks{}.csv'.format(str(week),str(master_info['Number_Vehicles'])))
